## Azure 云方案架构

Azure PowerShell 提供了可在 Azure 订阅中创建虚拟机的 New-AzVM 命令。 若要使用 Azure PowerShell，请启动 PowerShell 应用程序，然后发出如下命令

In [1]:
New-AzVm `
    -ResourceGroupName "CrmTestingResourceGroup" `
    -Name "CrmUnitTests" `
    -Image "UbuntuLTS"
    ...

Error: Command cancelled.

“Az”是 Azure PowerShell 模块的正式名称，其中包含用于处理 Azure 功能的 cmdlet。 它包含数百个 cmdlet，可以控制几乎每个 Azure 资源的所有方面。 可以使用以下功能和其他功能：

- 资源组
- 存储
- VM
- Azure AD
- 容器
- 机器学习

### 实验室方案
你是组织的 Azure 管理员。 你决定使用 Azure PowerShell 自动执行一些常见的管理任务。

- 创建资源组。
- 创建托管磁盘。
- 更改托管磁盘的配置。

### 目标
- 任务 1：在 Azure Cloud Shell 中启动 PowerShell 会话
- 任务 2：使用 PowerShell 创建资源组和托管磁盘。
   - 创建资源组。
- 在资源组中创建托管磁盘。
- 任务 3：使用 Azure PowerShell 配置托管磁盘。
   - 增加托管磁盘的大小。
   - 更改磁盘性能 SKU。

如何找到所需的特定命令？ 一种方法是使用 az find。 例如，如果要查找可能有助于管理存储 blob 的命令，可使用以下查找命令：

In [ ]:
az find blob

### 资源评审术语
- 资源 - 可通过 Azure 获取的可管理项。 部分常见资源包括虚拟机、存储帐户、Web 应用、数据库和虚拟网络，但这只是其中一小部分。
- 资源组 — 一个容器，用于保存 Azure 解决方案的相关资源。 资源组可以包含解决方案的所有资源，也可以只包含想要作为组来管理的资源。 根据对组织有利的原则，决定如何将资源分配到资源组。
- 资源提供程序 — 一种服务，提供可以通过 Resource Manager 进行部署和管理的资源。 每个资源提供程序提供用于处理所部署资源的操作。 部分常见资源提供程序包括 Microsoft.Compute（提供虚拟机资源）、Microsoft.-  - Storage（提供存储帐户资源）和 Microsoft.Web（提供与 Web 应用相关的资源）。
- 模板 - 一个 JavaScript 对象表示法 (JSON) 文件，用于定义一个或多个要部署到资源组的资源。 它也会定义所部署资源之间的依赖关系。 使用模板能够以一致方式反复部署资源。
- 声明性语法 — 一种语法，允许声明“以下是我想要创建的项目”，而不需要编写一系列编程命令来进行创建。 资源管理器模板便是声明性语法的其中一个示例。 在该文件中，可以定义要部署到 Azure 的基础结构的属性。

### 约束

- 每个资源只能在一个资源组中。
- 资源组无法重命名。
- 资源组可以有许多不同类型的资源（服务）。
- 资源组可以包含来自多个不同区域的资源。

###  创建资源组
定义资源组时，需要考虑以下几个重要因素：

- 组中的所有资源应该共享相同的生命周期。 一起部署、更新和删除这些资源。 如果某个资源（例如数据库服务器）需要采用不同的部署周期，则它应在另一个资源组中。
- 每个资源只能在一个资源组中。
- 随时可以在资源组添加或删除资源。
- 可以将资源从一个资源组移到另一个组。 限制适用于移动资源。
- 资源组可以包含位于不同区域的资源。
- 资源组可用于划分对管理操作的访问控制。
- 资源可与其他资源组中的资源进行交互。 如果两个资源相关，但不共享相同的生命周期，那么这种交互很常见（例如，Web 应用连接到数据库）。
创建资源组时，需要提供该资源组的位置。资源组存储有关资源的元数据。 因此，当指定资源组的位置时，也就指定了元数据的存储位置。 出于合规性原因，可能需要确保数据存储在某一特定区域。

### 创建 Azure 资源管理器锁
在 Azure 中预配资源的一个常见问题是可以轻松地将其删除。 只需几个步骤，过分热情或粗心的管理员就可能意外擦除几个月的工作。 组织可以使用资源管理器锁设置一个结构，以防止意外删除 Azure 中的资源。

可以将锁与订阅、资源组或资源关联起来。
锁由子资源继承。
#### 锁类型
有以下两种类型的资源锁。
> 只有“所有者”和“用户访问管理员”角色才能创建或删除管理锁。
- 只读锁，可防止对资源进行任何更改。
- 删除锁，可防止删除。

### 重新组织 Azure 资源

有时可能需要将资源移到新订阅或同一订阅的新资源组中。

在移动资源的过程中，源组和目标组都会锁定。 在完成移动之前，将阻止对资源组执行写入和删除操作。 此锁意味着将无法添加、更新或删除资源组中的资源。 锁定并不意味着资源不可用。 例如，即使将虚拟机移动到新的资源组，应用程序也仍可访问虚拟机。

### 删除资源和资源组

删除资源组时要格外小心。 删除资源组会删除其包含的所有资源。 该资源组可能包含其他资源组中的资源所依赖的资源。

使用 PowerShell 删除资源组
若要删除资源组，请使用 Remove-AzResourceGroup。 
#### 删除资源
也可以删除资源组中的单个资源。 例如，这里我们将删除虚拟网络。 你可以将资源移至另一个资源组，而不是删除。

In [ ]:
Remove-AzResourceGroup -Name "ContosoRG01"


Confirm
Are you sure you want to remove resource group 'ContosoRG01'
[Y] Yes  [N] No  [S] Suspend  [?] Help(default is 'Y')Y - Continue with only the next step of the operation.
N - Skip this operation and proceed with the next operation.
S - Pause the current pipeline and return to the command prompt. Type "exit" to resume the pipeline.
[Y] Yes  [N] No  [S] Suspend  [?] Help(default is 'Y')True


### 使用Azure 资源管理器模板
方案
你的公司需确保虚拟机部署在整个组织中保持一致。

可以使用 Azure 资源管理器模板来部署虚拟机等资源。

使用 Azure 资源管理器模板自动部署虚拟机 (VM)

- 修改 Azure 资源管理器模板。
- 从模板部署。
- 将部署另存为 Azure 资源管理器模板

#### 模板优点
- 模板可提高一致性。 资源管理器模板为你和其他人提供一种通用语言来描述部署。 无论使用何种工具或 SDK 部署模板，模板内的结构、格式和表达式都保持不变。
- 模板有助于表达复杂部署。 利用模板，可以正确的顺序部署多个资源。 例如，你不会想要在创建操作系统 (OS) 磁盘或网络接口之前部署虚拟机。 资源管理器会首先映射每个资源及其依赖的资源，并创建所依赖的资源。 依赖项映射有助于确保按正确的顺序执行部署。
- 模板可减少手动、易出错的任务。 手动创建和连接资源非常耗时，而且容易出错。 资源管理器可确保部署每次发生的方式相同。
- 模板是代码。 模板通过代码表达需求。 将模板视为一种可像其他所有软件一样进行分享、测试和版本控制的“基础结构即代码”。 此外，由于模板是代码，因此可创建可以跟踪的“书面记录”。 模板代码可记录部署。 大多数用户通过某些修订控制（如 Git）维护其模板。 更改模板时，其修订历史记录还会记录模板（和部署）怎样随着时间推移不断变化。
- 模板有利于重复使用。 模板可包含模板运行时填充的参数。 参数可以定义用户名或密码、域名等等。 使用模板参数，可以创建多个版本的基础结构（如过渡过程和生产过程），但同时仍可使用相同的模板。
- 模板可链接。 可以将资源管理器模板链接在一起，使模板本身模块化。 可以编写较小的模板，每个模板定义解决方案的一部分，然后将其合并，创建完整的系统。
- 模板简化业务流程。 只需部署模板即可部署所有资源。 通常，这需要执行多个操作。

####  模板结构
Azure 资源管理器模板以 JSON 编写，可以以文本形式表示存储为对象（例如虚拟机）的数据。 JSON 文档实质上是键值对的集合。 每个密钥都是一个字符串，其值可以是如下形式：

一个字符串
数字
布尔表达式
值列表
对象（即其他键值对的集合）
资源管理器模板可以包含使用 JSON 表示法表示但与 JSON 语言本身无关的部分
```
{
    "$schema": "http://schema.management.​azure.com/schemas/2019-04-01/deploymentTemplate.json#",​
    "contentVersion": "",​
    "parameters": {},​
    "variables": {},​
    "functions": [],​
    "resources": [],​
    "outputs": {}​
}
```

##### Azure 资源管理器模板参数

在模板的 parameters 节中，可以指定在部署资源时能够输入的值。 参数的可用属性为：
```
"parameters": {
    "<parameter-name>" : {
        "type" : "<type-of-parameter-value>",
        "defaultValue": "<default-value-of-parameter>",
        "allowedValues": [ "<array-of-allowed-values>" ],
        "minValue": <minimum-value-for-int>,
        "maxValue": <maximum-value-for-int>,
        "minLength": <minimum-length-for-string-or-array>,
        "maxLength": <maximum-length-for-string-or-array-parameters>,
        "metadata": {
        "description": "<description-of-the parameter>"
        }
    }
}
```
下面示例演示了两个参数：一个用于虚拟机的用户名，另一个用于它的密码：

``` 
"parameters": {
  "adminUsername": {
    "type": "string",
    "metadata": {
      "description": "Username for the Virtual Machine."
    }
  },
  "adminPassword": {
    "type": "securestring",
    "metadata": {
      "description": "Password for the Virtual Machine."
    }
  }
}
```
##### Bicep 模板

Azure Bicep 是一种特定于域的语言 (DSL)，使用声明性语法来部署 Azure 资源。 它提供简明的语法、可靠的类型安全性以及对代码重用的支持。

可以使用 Bicep 而非 JSON 来开发 Azure 资源管理器模板（ARM 模板）。 用于创建 ARM 模板的 JSON 语法可能比较繁琐，需要使用复杂的表达式。 Bicep 语法降低了这种复杂性，改进了开发体验。 Bicep 是基于 ARM 模板 JSON 的透明抽象，不会丢失任何 JSON 模板功能。

将一个或一系列资源部署到 Azure 时，Bicep 中内置的工具可将 Bicep 模板转换为 JSON 模板。 此过程称为转译。 转译是将使用一种语言编写的源代码转换为另一种语言的过程。

Bicep 为模板创作提供了许多通过 JSON 的改进，包括：

- 更简单的语法：Bicep 为编写模板提供了更简单的语法。 可以直接引用参数和变量，而无需使用复杂的函数。 字符串内插用于替换串联来合并名称和其他项的值。 可以使用资源的符号名称（而不是复杂的引用语句）直接引用资源的属性。 这些语法改进有助于创作和读取 Bicep 模板。

- 模块：可以将复杂模板部署分解为更小的模块文件，并在主模板中引用它们。 这些模块提供更轻松的管理和更大的可重用性。

- 自动依赖项管理：在大多数情况下，Bicep 会自动检测资源之间的依赖关系。 此过程将删除模板创作中涉及的一些工作。

- 类型验证和 IntelliSense：适用于 Visual Studio Code 的 Bicep 扩展为所有 Azure 资源类型 API 定义提供丰富的验证和 IntelliSense。 此功能有助于提供更轻松的创作体验。

##### 快速入门模板 
部分模板提供了部署解决方案所需的一切，其他模板则可能是你的模板的基础。 不管怎样，都可以学习这些模板，了解如何更好地创作和生成自己的模板。

README.md 文件概述了模板的功能。
azuredeploy.json 文件定义将部署的资源。
azuredeploy.parameters.json 文件提供模板所需的值。


#### 如何使用 Azure PowerShell 创建资源组
安装 Azure 模块后，可以开始使用 Azure。 让我们执行一个常见任务：创建资源组。 我们知道，使用资源组可以统一管理相关的资源。 创建新资源组是启动新 Azure 解决方案时执行的首要任务之一。

需要执行四个步骤：

1. 导入 Azure cmdlet。
2. 连接到 Azure 订阅。
3. 创建资源组。
4. 验证创建是否成功。

##### 连接
使用 Azure PowerShell 的本地安装时，在执行 Azure 命令之前需要进行身份验证。 Connect-AzAccount cmdlet 会提示输入 Azure 凭据，然后连接到 Azure 订阅。 它具有许多可选参数，但如果你只需要交互式提示，则不需要任何参数：


In [ ]:
Connect-AzAccount

Error: Command cancelled.

##### 使用订阅
如果你是 Azure 的新手，也许只有一个订阅。 但如果你使用 Azure 有一段时间，可能已创建了多个 Azure 订阅。 可将 Azure PowerShell 配置为针对特定的订阅执行命令。

每次只能在一个订阅中操作。 使用 Get-AzContext cmdlet 确定哪个订阅处于活动状态。 如果它不正确，可以使用另一个 cmdlet 更改订阅。

使用 Get-AzSubscription 命令获取帐户中所有订阅名称的列表。

通过传递要选择的订阅名称来更改订阅。

In [ ]:
Get-AzSubscription


In [ ]:
Set-AzContext -Subscription '00000000-0000-0000-0000-000000000000'

为使视图显得更简洁，可以使用竖线“|”将 Get-AzResourceGroup 的输出发送到 Format-Table cmdlet。
Azure PowerShell

In [ ]:
Get-AzResourceGroup | Format-Table